In [ ]:
!pip install neuralforecast

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.6/170.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 10.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import torch
from typing import Optional, Union, Tuple


import torch.nn.functional as F
from torch.distributions import Distribution
from torch.distributions import Bernoulli, Normal, StudentT, Poisson, NegativeBinomial

from torch.distributions import constraints


from neuralforecast import NeuralForecast
from neuralforecast.models import NHITS
from neuralforecast.tsdataset import TimeSeriesDataset
from neuralforecast.utils import AirPassengers, AirPassengersPanel, AirPassengersStatic
from neuralforecast.losses.numpy import rmse, mape
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
from neuralforecast.losses.pytorch import MQLoss, DistributionLoss, MSE, MAE

In [ ]:
from numpy.random import seed
from random import randrange

In [ ]:
df = pd.read_excel("SP100_vol.xlsx", index_col="Date",parse_dates=True)
columns=df.columns
df_static=pd.read_excel("Static_df.xlsx")
n_series=len(df.columns)
train = df.iloc[:int(len(df['NKE'])*0.7)]
validation_length=int(len(df['NKE'])*0.7)-int(len(df['NKE'])*0.5)
train=train.unstack().reset_index(name="Value").rename(columns={"level_0":"unique_id", "Date":"ds", "Value":"y"})

In [ ]:
n_inputs = [3,5,10,15,21,42,84]
mlp_units = [[[712, 712], [712, 712]],[[512, 512], [512, 512]],[[250, 250], [250, 250]],[[100, 100], [100, 100]]]
epochs=[50,100,150,250,350,450,550,650,750]
learning_rate=[0.0005,0.0001,0.00005,0.00001]
num_lr_decays=[5,3,2,1]
scaler_type=["robust","standard",'minmax']
losses=[MSE(),MAE(),MQLoss(level=[80, 90]),DistributionLoss(distribution='StudentT', level=[80, 90])]
n_blocks=[[1, 1],[2, 2],[3, 3],[5, 5]]
dropout_prob_theta=[0, 0.2, 0.3, 0.4]
n_pool_kernel_size=[[2, 2, 1], [4, 2, 1], [4, 4, 2]]
pooling_mode=["MaxPool1d", "AvgPool1d"]
interpolation_mode=['linear', 'nearest']

# Hyperparameter Search

In [ ]:
for z in range (5):
  i=randrange(7)
  h=randrange(4)
  m=randrange(4)
  a=randrange(4)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)
  a1=randrange(4)
  a2=randrange(3)
  a3=randrange(2)
  a4=randrange(2)
  model = NHITS(h=1, input_size=n_inputs[i],
                loss=losses[m],
                scaler_type=scaler_type[o],
                learning_rate=learning_rate[n],
                stack_types=['identity','identity'],
                n_blocks=n_blocks[a],
                mlp_units=mlp_units[h],
                windows_batch_size=32,
                num_lr_decays=num_lr_decays[n],
                val_check_steps=500,
                stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
                max_steps=epochs[k],
                dropout_prob_theta = dropout_prob_theta[a1],
                n_pool_kernel_size=n_pool_kernel_size[a2],
                pooling_mode=pooling_mode[a3],
                interpolation_mode=interpolation_mode[a4],
                random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "NHITS-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["NHITS"].values
  else:
    Y_hat=forecasts["NHITS-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | MLP units: {mlp_units[h]} | Loss: {losses[m]} | Scaler Type: {scaler_type[o]} | Interpolation: {interpolation_mode[a4]}")
  print(f"Learning Rate: {n} | Block: {n_blocks[a]} | Dropout: {dropout_prob_theta[a1]} | Pooling: {pooling_mode[a3]} | Kernel: {n_pool_kernel_size[a2]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 59107986


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Epoch: 100 | Inputs: 3 | MLP units: [[100, 100], [100, 100]] | Loss: DistributionLoss() | Scaler Type: standard | Interpolation: nearest
Learning Rate: 1 | Block: [3, 3] | Dropout: 0.4 | Pooling: MaxPool1d | Kernel: [2, 2, 1]
RMSE: 0.002764413591381078 | QLIKE: 0.02829557102123029 | MAE: 0.001454058037620249 | MAPE: 0.14401676939646418


In [ ]:
for z in range (5):
  i=randrange(7)
  h=randrange(4)
  m=randrange(4)
  a=randrange(4)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)
  a1=randrange(4)
  a2=randrange(3)
  a3=randrange(2)
  a4=randrange(2)
  model = NHITS(h=1, input_size=n_inputs[i],
                loss=losses[m],
                scaler_type=scaler_type[o],
                learning_rate=learning_rate[n],
                stack_types=['identity','identity'],
                n_blocks=n_blocks[a],
                mlp_units=mlp_units[h],
                windows_batch_size=32,
                num_lr_decays=num_lr_decays[n],
                val_check_steps=500,
                stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
                max_steps=epochs[k],
                dropout_prob_theta = dropout_prob_theta[a1],
                n_pool_kernel_size=n_pool_kernel_size[a2],
                pooling_mode=pooling_mode[a3],
                interpolation_mode=interpolation_mode[a4],
                random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "NHITS-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["NHITS"].values
  else:
    Y_hat=forecasts["NHITS-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | MLP units: {mlp_units[h]} | Loss: {losses[m]} | Scaler Type: {scaler_type[o]} | Interpolation: {interpolation_mode[a4]}")
  print(f"Learning Rate: {n} | Block: {n_blocks[a]} | Dropout: {dropout_prob_theta[a1]} | Pooling: {pooling_mode[a3]} | Kernel: {n_pool_kernel_size[a2]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 69220072


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Epoch: 50 | Inputs: 5 | MLP units: [[100, 100], [100, 100]] | Loss: MAE() | Scaler Type: standard | Interpolation: linear
Learning Rate: 2 | Block: [3, 3] | Dropout: 0.3 | Pooling: MaxPool1d | Kernel: [4, 4, 2]
RMSE: 0.002781363603031141 | QLIKE: 0.02815873350633971 | MAE: 0.001446136466298664 | MAPE: 0.14402319393307303


In [ ]:
for z in range (5):
  i=randrange(7)
  h=randrange(4)
  m=randrange(4)
  a=randrange(4)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)
  a1=randrange(4)
  a2=randrange(3)
  a3=randrange(2)
  a4=randrange(2)
  model = NHITS(h=1, input_size=n_inputs[i],
                loss=losses[m],
                scaler_type=scaler_type[o],
                learning_rate=learning_rate[n],
                stack_types=['identity','identity'],
                n_blocks=n_blocks[a],
                mlp_units=mlp_units[h],
                windows_batch_size=32,
                num_lr_decays=num_lr_decays[n],
                val_check_steps=500,
                stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
                max_steps=epochs[k],
                dropout_prob_theta = dropout_prob_theta[a1],
                n_pool_kernel_size=n_pool_kernel_size[a2],
                pooling_mode=pooling_mode[a3],
                interpolation_mode=interpolation_mode[a4],
                random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "NHITS-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["NHITS"].values
  else:
    Y_hat=forecasts["NHITS-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | MLP units: {mlp_units[h]} | Loss: {losses[m]} | Scaler Type: {scaler_type[o]} | Interpolation: {interpolation_mode[a4]}")
  print(f"Learning Rate: {n} | Block: {n_blocks[a]} | Dropout: {dropout_prob_theta[a1]} | Pooling: {pooling_mode[a3]} | Kernel: {n_pool_kernel_size[a2]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 6589365


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Epoch: 450 | Inputs: 21 | MLP units: [[512, 512], [512, 512]] | Loss: DistributionLoss() | Scaler Type: minmax | Interpolation: linear
Learning Rate: 1 | Block: [1, 1] | Dropout: 0.2 | Pooling: MaxPool1d | Kernel: [2, 2, 1]
RMSE: 0.0027040353019622144 | QLIKE: 0.02712200475317952 | MAE: 0.001631905411313709 | MAPE: 0.15554612373141502


In [ ]:
for z in range (5):
  i=randrange(7)
  h=randrange(4)
  m=randrange(4)
  a=randrange(4)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)
  a1=randrange(4)
  a2=randrange(3)
  a3=randrange(2)
  a4=randrange(2)
  model = NHITS(h=1, input_size=n_inputs[i],
                loss=losses[m],
                scaler_type=scaler_type[o],
                learning_rate=learning_rate[n],
                stack_types=['identity','identity'],
                n_blocks=n_blocks[a],
                mlp_units=mlp_units[h],
                windows_batch_size=32,
                num_lr_decays=num_lr_decays[n],
                val_check_steps=500,
                stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
                max_steps=epochs[k],
                dropout_prob_theta = dropout_prob_theta[a1],
                n_pool_kernel_size=n_pool_kernel_size[a2],
                pooling_mode=pooling_mode[a3],
                interpolation_mode=interpolation_mode[a4],
                random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "NHITS-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["NHITS"].values
  else:
    Y_hat=forecasts["NHITS-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | MLP units: {mlp_units[h]} | Loss: {losses[m]} | Scaler Type: {scaler_type[o]} | Interpolation: {interpolation_mode[a4]}")
  print(f"Learning Rate: {n} | Block: {n_blocks[a]} | Dropout: {dropout_prob_theta[a1]} | Pooling: {pooling_mode[a3]} | Kernel: {n_pool_kernel_size[a2]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 77483894


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Epoch: 350 | Inputs: 15 | MLP units: [[512, 512], [512, 512]] | Loss: DistributionLoss() | Scaler Type: minmax | Interpolation: linear
Learning Rate: 0 | Block: [2, 2] | Dropout: 0 | Pooling: AvgPool1d | Kernel: [4, 4, 2]
RMSE: 0.002765160590040289 | QLIKE: 0.028323861640038153 | MAE: 0.0015386054797144675 | MAPE: 0.15260393735960462


In [ ]:
for z in range (5):
  i=randrange(7)
  h=randrange(4)
  m=randrange(4)
  a=randrange(4)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)
  a1=randrange(4)
  a2=randrange(3)
  a3=randrange(2)
  a4=randrange(2)
  model = NHITS(h=1, input_size=n_inputs[i],
                loss=losses[m],
                scaler_type=scaler_type[o],
                learning_rate=learning_rate[n],
                stack_types=['identity','identity'],
                n_blocks=n_blocks[a],
                mlp_units=mlp_units[h],
                windows_batch_size=32,
                num_lr_decays=num_lr_decays[n],
                val_check_steps=500,
                stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
                max_steps=epochs[k],
                dropout_prob_theta = dropout_prob_theta[a1],
                n_pool_kernel_size=n_pool_kernel_size[a2],
                pooling_mode=pooling_mode[a3],
                interpolation_mode=interpolation_mode[a4],
                random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "NHITS-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["NHITS"].values
  else:
    Y_hat=forecasts["NHITS-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | MLP units: {mlp_units[h]} | Loss: {losses[m]} | Scaler Type: {scaler_type[o]} | Interpolation: {interpolation_mode[a4]}")
  print(f"Learning Rate: {n} | Block: {n_blocks[a]} | Dropout: {dropout_prob_theta[a1]} | Pooling: {pooling_mode[a3]} | Kernel: {n_pool_kernel_size[a2]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 44502625


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Epoch: 100 | Inputs: 84 | MLP units: [[512, 512], [512, 512]] | Loss: MQLoss() | Scaler Type: robust | Interpolation: nearest
Learning Rate: 0 | Block: [3, 3] | Dropout: 0.3 | Pooling: MaxPool1d | Kernel: [2, 2, 1]
RMSE: 0.0027156843867745723 | QLIKE: 0.027579871738127495 | MAE: 0.0016434552434353086 | MAPE: 0.16028796096309428


In [ ]:
for z in range (5):
  i=randrange(7)
  h=randrange(4)
  m=randrange(4)
  a=randrange(4)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)
  a1=randrange(4)
  a2=randrange(3)
  a3=randrange(2)
  a4=randrange(2)
  model = NHITS(h=1, input_size=n_inputs[i],
                loss=losses[m],
                scaler_type=scaler_type[o],
                learning_rate=learning_rate[n],
                stack_types=['identity','identity'],
                n_blocks=n_blocks[a],
                mlp_units=mlp_units[h],
                windows_batch_size=32,
                num_lr_decays=num_lr_decays[n],
                val_check_steps=500,
                stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
                max_steps=epochs[k],
                dropout_prob_theta = dropout_prob_theta[a1],
                n_pool_kernel_size=n_pool_kernel_size[a2],
                pooling_mode=pooling_mode[a3],
                interpolation_mode=interpolation_mode[a4],
                random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "NHITS-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["NHITS"].values
  else:
    Y_hat=forecasts["NHITS-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | MLP units: {mlp_units[h]} | Loss: {losses[m]} | Scaler Type: {scaler_type[o]} | Interpolation: {interpolation_mode[a4]}")
  print(f"Learning Rate: {n} | Block: {n_blocks[a]} | Dropout: {dropout_prob_theta[a1]} | Pooling: {pooling_mode[a3]} | Kernel: {n_pool_kernel_size[a2]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 122760049


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Epoch: 250 | Inputs: 5 | MLP units: [[250, 250], [250, 250]] | Loss: MSE() | Scaler Type: robust | Interpolation: nearest
Learning Rate: 2 | Block: [1, 1] | Dropout: 0.3 | Pooling: AvgPool1d | Kernel: [4, 4, 2]
RMSE: 0.002805832538903971 | QLIKE: 0.027073198988484495 | MAE: 0.0014415187149189356 | MAPE: 0.14545941826992032


In [ ]:
for z in range (5):
  i=randrange(7)
  h=randrange(4)
  m=randrange(4)
  a=randrange(4)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)
  a1=randrange(4)
  a2=randrange(3)
  a3=randrange(2)
  a4=randrange(2)
  model = NHITS(h=1, input_size=n_inputs[i],
                loss=losses[m],
                scaler_type=scaler_type[o],
                learning_rate=learning_rate[n],
                stack_types=['identity','identity'],
                n_blocks=n_blocks[a],
                mlp_units=mlp_units[h],
                windows_batch_size=32,
                num_lr_decays=num_lr_decays[n],
                val_check_steps=500,
                stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
                max_steps=epochs[k],
                dropout_prob_theta = dropout_prob_theta[a1],
                n_pool_kernel_size=n_pool_kernel_size[a2],
                pooling_mode=pooling_mode[a3],
                interpolation_mode=interpolation_mode[a4],
                random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "NHITS-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["NHITS"].values
  else:
    Y_hat=forecasts["NHITS-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | MLP units: {mlp_units[h]} | Loss: {losses[m]} | Scaler Type: {scaler_type[o]} | Interpolation: {interpolation_mode[a4]}")
  print(f"Learning Rate: {n} | Block: {n_blocks[a]} | Dropout: {dropout_prob_theta[a1]} | Pooling: {pooling_mode[a3]} | Kernel: {n_pool_kernel_size[a2]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 93471300


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Epoch: 750 | Inputs: 5 | MLP units: [[100, 100], [100, 100]] | Loss: DistributionLoss() | Scaler Type: robust | Interpolation: nearest
Learning Rate: 3 | Block: [1, 1] | Dropout: 0.2 | Pooling: MaxPool1d | Kernel: [4, 2, 1]
RMSE: 0.0028061639243976986 | QLIKE: 0.027259903124337806 | MAE: 0.0014395587809333161 | MAPE: 0.14457959604807893


In [ ]:
for z in range (5):
  i=randrange(7)
  h=randrange(4)
  m=randrange(4)
  a=randrange(4)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)
  a1=randrange(4)
  a2=randrange(3)
  a3=randrange(2)
  a4=randrange(2)
  model = NHITS(h=1, input_size=n_inputs[i],
                loss=losses[m],
                scaler_type=scaler_type[o],
                learning_rate=learning_rate[n],
                stack_types=['identity','identity'],
                n_blocks=n_blocks[a],
                mlp_units=mlp_units[h],
                windows_batch_size=32,
                num_lr_decays=num_lr_decays[n],
                val_check_steps=500,
                stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
                max_steps=epochs[k],
                dropout_prob_theta = dropout_prob_theta[a1],
                n_pool_kernel_size=n_pool_kernel_size[a2],
                pooling_mode=pooling_mode[a3],
                interpolation_mode=interpolation_mode[a4],
                random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "NHITS-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["NHITS"].values
  else:
    Y_hat=forecasts["NHITS-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | MLP units: {mlp_units[h]} | Loss: {losses[m]} | Scaler Type: {scaler_type[o]} | Interpolation: {interpolation_mode[a4]}")
  print(f"Learning Rate: {n} | Block: {n_blocks[a]} | Dropout: {dropout_prob_theta[a1]} | Pooling: {pooling_mode[a3]} | Kernel: {n_pool_kernel_size[a2]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 58832951


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Epoch: 650 | Inputs: 21 | MLP units: [[250, 250], [250, 250]] | Loss: MAE() | Scaler Type: standard | Interpolation: linear
Learning Rate: 3 | Block: [3, 3] | Dropout: 0.4 | Pooling: AvgPool1d | Kernel: [4, 2, 1]
RMSE: 0.00272945627831789 | QLIKE: 0.027006931483143988 | MAE: 0.0015017116729681647 | MAPE: 0.1481451512059265


INFO:lightning_fabric.utilities.seed:Seed set to 77483894

Epoch: 150 | Inputs: 21 | MLP units: [[712, 712], [712, 712]] | Loss: DistributionLoss() | Scaler Type: standard | Interpolation: nearest
Learning Rate: 0 | Block: [2, 2] | Dropout: 0 | Pooling: MaxPool1d | Kernel: [4, 2, 1]

RMSE: 0.00261795408412616 | QLIKE: 0.024710279430817574 | MAE: 0.0016030544887469216 | MAPE: 0.15921628282272618

In [ ]:
for z in range (5):
  randomseed=randrange(129228148)
  model = NHITS(h=1, input_size=21,
                loss=losses[3],
                scaler_type='standard',
                learning_rate=learning_rate[0],
                stack_types=['identity','identity'],
                n_blocks=[2,2],
                mlp_units= [[712, 712], [712, 712]],
                windows_batch_size=32,
                num_lr_decays=num_lr_decays[0],
                val_check_steps=500,
                stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
                max_steps=150,
                dropout_prob_theta = 0,
                n_pool_kernel_size=[4,2,1],
                pooling_mode='MaxPool1d' ,
                interpolation_mode='nearest',
                random_seed=randomseed)
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "NHITS-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["NHITS"].values
  else:
    Y_hat=forecasts["NHITS-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Random Seed: {randomseed}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 75404644


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Random Seed: 60225863
RMSE: 0.0026655954642769986 | QLIKE: 0.025489856683519704 | MAE: 0.001593193720650095 | MAPE: 0.15188960218351394


In [ ]:
for z in range (5):
  randomseed=randrange(129228148)
  model = NHITS(h=1, input_size=21,
                loss=losses[3],
                scaler_type='standard',
                learning_rate=learning_rate[0],
                stack_types=['identity','identity'],
                n_blocks=[2,2],
                mlp_units= [[712, 712], [712, 712]],
                windows_batch_size=32,
                num_lr_decays=num_lr_decays[0],
                val_check_steps=500,
                stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
                max_steps=150,
                dropout_prob_theta = 0,
                n_pool_kernel_size=[4,2,1],
                pooling_mode='MaxPool1d' ,
                interpolation_mode='nearest',
                random_seed=randomseed)
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "NHITS-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["NHITS"].values
  else:
    Y_hat=forecasts["NHITS-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Random Seed: {randomseed}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 123422336


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Random Seed: 30530965
RMSE: 0.0026524392654564814 | QLIKE: 0.02519682319643343 | MAE: 0.0015939968525946477 | MAPE: 0.15299752887304482


In [ ]:
for z in range (5):
  randomseed=randrange(129228148)
  model = NHITS(h=1, input_size=21,
                loss=losses[3],
                scaler_type='standard',
                learning_rate=learning_rate[0],
                stack_types=['identity','identity'],
                n_blocks=[2,2],
                mlp_units= [[712, 712], [712, 712]],
                windows_batch_size=32,
                num_lr_decays=num_lr_decays[0],
                val_check_steps=500,
                stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
                max_steps=150,
                dropout_prob_theta = 0,
                n_pool_kernel_size=[4,2,1],
                pooling_mode='MaxPool1d' ,
                interpolation_mode='nearest',
                random_seed=randomseed)
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "NHITS-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["NHITS"].values
  else:
    Y_hat=forecasts["NHITS-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Random Seed: {randomseed}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 100965809


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Random Seed: 85634168
RMSE: 0.002735827403816384 | QLIKE: 0.026847192707995446 | MAE: 0.0016339817088380948 | MAPE: 0.1540480289438204


In [ ]:
for z in range (5):
  randomseed=randrange(129228148)
  model = NHITS(h=1, input_size=21,
                loss=losses[3],
                scaler_type='standard',
                learning_rate=learning_rate[0],
                stack_types=['identity','identity'],
                n_blocks=[2,2],
                mlp_units= [[712, 712], [712, 712]],
                windows_batch_size=32,
                num_lr_decays=num_lr_decays[0],
                val_check_steps=500,
                stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
                max_steps=150,
                dropout_prob_theta = 0,
                n_pool_kernel_size=[4,2,1],
                pooling_mode='MaxPool1d' ,
                interpolation_mode='nearest',
                random_seed=randomseed)
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "NHITS-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["NHITS"].values
  else:
    Y_hat=forecasts["NHITS-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Random Seed: {randomseed}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 63738212


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Random Seed: 75075575
RMSE: 0.0026600858096814625 | QLIKE: 0.025176456202993296 | MAE: 0.0016388522072901474 | MAPE: 0.1609501859245695


# Test

In [ ]:
test_length=int(len(df['NKE'])*0.3)
df1=df.unstack().reset_index(name="Value").rename(columns={"level_0":"unique_id", "Date":"ds", "Value":"y"})

In [ ]:
model = NHITS(h=1, input_size=21,
                loss=losses[3],
                scaler_type='standard',
                learning_rate=learning_rate[0],
                stack_types=['identity','identity'],
                n_blocks=[2,2],
                mlp_units= [[712, 712], [712, 712]],
                windows_batch_size=32,
                num_lr_decays=num_lr_decays[0],
                val_check_steps=500,
                stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
                max_steps=150,
                dropout_prob_theta = 0,
                n_pool_kernel_size=[4,2,1],
                pooling_mode='MaxPool1d' ,
                interpolation_mode='nearest',
              random_seed=123422336)
fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
forecasts = fcst.cross_validation(df=df1,val_size=1,static_df=df_static,n_windows=None, test_size=test_length-test_length%1,step_size=1)
forecasts = forecasts.dropna()
if "NHITS-median" not in list(forecasts.columns.values):
  Y_hat=forecasts["NHITS"].values
else:
  Y_hat=forecasts["NHITS-median"].values
Y_true=forecasts["y"].values
RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 123422336


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

RMSE: 0.004007353152580852 | QLIKE: 0.03299555886615222 | MAE: 0.0024607141400636353 | MAPE: 0.1840706474505275


In [ ]:
if "NHITS-median" not in list(forecasts.columns.values):
  df2=forecasts.drop(columns=["cutoff", "y"]).set_index("ds")
  df2=df2.pivot(columns="unique_id")
else:
  df2=forecasts[["ds","unique_id","NHITS-median"]].set_index("ds")
  df2=df2.pivot(columns="unique_id")

In [ ]:
df2.to_excel("NHITS forecasts.xlsx")